# Transformer ile Metin Duygu Analizi (Sentiment Analysis)


In [ ]:
import pandas as pd

# Küçük bir duygu analizi veri seti
data = {
    "text": [
        "Bu film harikaydı, tekrar izlemek istiyorum!",
        "Çok kötü bir deneyimdi, hiç beğenmedim.",
        "Mükemmel bir ürün, herkese tavsiye ederim.",
        "Zaman kaybıydı, hiç memnun kalmadım.",
        "Gerçekten çok güzel bir yer, manzara harikaydı.",
        "Hayatımda gördüğüm en berbat hizmetti.",
        "Tadı çok lezzetliydi, tekrar alacağım.",
        "Berbattı, asla tekrar denemem."
    ],
    "label": [1, 0, 1, 0, 1, 0, 1, 0]  # 1: Olumlu, 0: Olumsuz
}

df = pd.DataFrame(data)

# Veri setini gösterelim
from IPython.display import display
display(df)
print("Duygu Analizi Veri Seti")

,text,label
0,"Bu film harikaydı, tekrar izlemek istiyorum!",1
1,"Çok kötü bir deneyimdi, hiç beğenmedim.",0
2,"Mükemmel bir ürün, herkese tavsiye ederim.",1
3,"Zaman kaybıydı, hiç memnun kalmadım.",0
4,"Gerçekten çok güzel bir yer, manzara harikaydı.",1
5,Hayatımda gördüğüm en berbat hizmetti.,0
6,"Tadı çok lezzetliydi, tekrar alacağım.",1
7,"Berbattı, asla tekrar denemem.",0


Duygu Analizi Veri Seti


In [ ]:
!pip install transformers datasets torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os
os.environ["WANDB_DISABLED"] = "true"


# Model ve tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Veriyi token haline getirme
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

dataset = SentimentDataset(df["text"].tolist(), df["label"].tolist())

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Modeli eğitme
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=12, training_loss=0.6621385018030802, metrics={'train_runtime': 35.7857, 'train_samples_per_second': 0.671, 'train_steps_per_second': 0.335, 'total_flos': 271333275840.0, 'train_loss': 0.6621385018030802, 'epoch': 3.0})

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Olumlu" if prediction == 1 else "Olumsuz"

test_sentence = "Tadı çok lezzetliydi, tekrar alacağım."
print(f"Model Tahmini: {predict_sentiment(test_sentence)}")


Model Tahmini: Olumlu


# Transformer ile Dil Çevirisi(Translation - English to French)

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Modeli ve Tokenizer'ı yükleyelim
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
sentences = [
    "The weather is nice today.",
    "I love machine learning.",
    "This is a great opportunity for you.",
    "Can you help me with this task?",
    "I am learning Transformers."
]


In [ ]:
def translate_to_french(text):
    input_text = "translate English to French: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)

    # Model ile çeviri yapalım
    output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Tokenleri metne çevirelim
    translated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return translated_text


In [ ]:
translations = {sentence: translate_to_french(sentence) for sentence in sentences}

import pandas as pd
# Mevcut olmayan 'ace_tools' yerine, IPython.display'den display fonksiyonunu kullanıyoruz
# ve özel bir başlık veriyoruz.
from IPython.display import display
df_translations = pd.DataFrame(translations.items(), columns=["English", "French"])
display(df_translations)
print("Çeviri Sonuçları")


,English,French
0,The weather is nice today.,Le temps est agréable aujourd'hui.
1,I love machine learning.,J'aime l'apprentissage de la machine.
2,This is a great opportunity for you.,C'est une excellente occasion pour vous.
3,Can you help me with this task?,Pouvez-vous m'aider dans cette tâche?
4,I am learning Transformers.,Je suis en train d'apprendre Transformers.


Çeviri Sonuçları


# Transformer ile Metin Özetleme (Text Summarization)

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Modeli ve Tokenizer'ı yükleyelim
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
text = """
Yapay zeka, bilgisayar sistemlerinin insan benzeri zekaya sahip olmasını amaçlayan bir bilim dalıdır.
Bu alan; makine öğrenmesi, doğal dil işleme, görüntü işleme ve derin öğrenme gibi alt dalları içerir.
Son yıllarda, yapay zeka teknolojileri büyük ilerlemeler kaydetmiş ve sağlık, finans, otomotiv gibi birçok sektörde yaygın şekilde kullanılmaya başlanmıştır.
Özellikle derin öğrenme ve büyük veri teknolojilerinin gelişmesi, yapay zekanın daha akıllı hale gelmesini sağlamıştır.
Ancak, bu teknolojinin etik ve güvenlik konularında bazı sorunları da bulunmaktadır.
"""


In [ ]:
def summarize_text(text):
    input_text = "summarize: " + text  # Modelin anlaması için önüne "summarize:" ekliyoruz.
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Model özetleme yapıyor
    output_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

    # Tokenleri metne dönüştürelim
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return summary


In [ ]:
summary_result = summarize_text(text)

import pandas as pd
# Mevcut olmayan 'ace_tools' yerine, IPython.display'den display fonksiyonunu kullanıyoruz
# ve özel bir başlık veriyoruz.
from IPython.display import display

df_summary = pd.DataFrame({"Original Text": [text], "Summary": [summary_result]})
display(df_summary)
print("Metin Özetleme Sonuçları") # Başlığı bu şekilde ekliyoruz


,Original Text,Summary
0,"\nYapay zeka, bilgisayar sistemlerinin insan b...",bilgisayar sistemlerinin insan benzeri zekaya ...


Metin Özetleme Sonuçları


# Transformer ile Soru-Cevaplama (Question Answering)

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import pipeline

# Question Answering için Transformer modeli
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
context = """
Albert Einstein, 14 Mart 1879'da Almanya'nın Ulm kentinde doğdu.
Teorik fizikçi olan Einstein, görelilik teorisi ile ünlüdür ve 1921'de Nobel Fizik Ödülü kazanmıştır.
E = mc^2 denklemi, onun en çok bilinen çalışmalarından biridir.
1933 yılında Almanya'dan ayrılarak ABD'ye göç etmiş ve burada Princeton Üniversitesi'nde çalışmıştır.
Türkiye'nin başkenti Ankara'dır.
"""


In [ ]:
questions = [
    "Einstein hangi yıl doğdu?",
    "En ünlü denklemi nedir?",
    "Hangi üniversitede çalıştı?",
    "Türkiye'nin başkenti neresidir?"
]

# Modeli kullanarak cevapları alalım
answers = [qa_pipeline(question=q, context=context)["answer"] for q in questions]

import pandas as pd
# Mevcut olmayan 'ace_tools' yerine, IPython.display'den display fonksiyonunu kullanıyoruz
# ve özel bir başlık veriyoruz.
from IPython.display import display

df_qa = pd.DataFrame({"Question": questions, "Answer": answers})
display(df_qa)
print("Soru-Cevap Sonuçları") # Başlığı bu şekilde ekliyoruz

,Question,Answer
0,Einstein hangi yıl doğdu?,Ulm kentinde doğdu
1,En ünlü denklemi nedir?,mc^2
2,Hangi üniversitede çalıştı?,Princeton
3,Türkiye'nin başkenti neresidir?,Ankara'dır


Soru-Cevap Sonuçları


# Transformer ile Kod Üretme (Code Generation)

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model ve Tokenizer'ı yükleyelim
model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [ ]:
def generate_code(prompt, max_length=200):
    """
    This function generates Python code based on the given prompt using the
    Salesforce/codegen-350M-mono model.
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    generated_ids = model.generate(input_ids, max_length=max_length, temperature=0.7, do_sample=True)
    generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return generated_code


In [ ]:
prompt = """Python'da bir fonksiyon yaz. Bu fonksiyon, verilen bir listenin içindeki çift sayıları filtreleyerek döndürmelidir.

# Örnek Kullanım:
# Input: [1, 2, 3, 4, 5, 6]
# Output: [2, 4, 6]

def filter_even_numbers(numbers):"""

generated_code = generate_code(prompt)

print("🔹 **Oluşturulan Python Kodu:**")
print(generated_code)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 **Oluşturulan Python Kodu:**
Python'da bir fonksiyon yaz. Bu fonksiyon, verilen bir listenin içindeki çift sayıları filtreleyerek döndürmelidir.

# Örnek Kullanım:
# Input: [1, 2, 3, 4, 5, 6]
# Output: [2, 4, 6]

def filter_even_numbers(numbers):
    for number in numbers:
        if number % 2 == 0:
            pass
        else:
            numbers.remove(number)
    return numbers


def filter_even_numbers_v3(numbers):
    return [number for number in numbers if number % 2 == 0]


print(filter_even_numbers([1, 2, 3, 4, 5, 6]))
print(filter_even


# Transformer ile Görsel Açıklama Üretme  (Image Captioning)

In [ ]:
!pip install transformers torch pillow


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image

# Model ve işleyiciyi (processor) yükleyelim
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)


In [ ]:
from IPython.display import display
import requests

# Örnek bir resim URL’si
image_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"

# Resmi indir ve aç
image = Image.open(requests.get(image_url, stream=True).raw)

# Resmi göster
display(image)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# Modelin ürettiği açıklamayı alalım
caption_result = generate_caption(image)

print("🔹 **Üretilen Açıklama:**")
print(caption_result)


🔹 **Üretilen Açıklama:**
a woman sitting on the beach with her dog


# Transformer ile Konuşma Tanıma (Speech-to-Text)

In [ ]:
!pip install transformers torch sounddevice scipy numpy librosa soundfile

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# Model ve Tokenizer'ı yükleyelim
model_name = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install sounddevice
!sudo apt-get install libportaudio-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libportaudio-dev is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'libportaudio-dev' has no installation candidate


In [ ]:
!apt-get update
!apt-get install -y libportaudio2

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Get:13 http://security.ubuntu.com/ubun

In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

# Ses kaydı için ayarlar
sample_rate = 16000  # Modelin beklediği örnekleme oranı
duration = 5  # Kaydı başlatınca kaç saniye devam edeceğini burada belirtebilirsin

def record_audio():
    print("🔴 **Kayıt başladı... Konuşabilirsiniz!**")

    # Available audio devices
    print(sd.query_devices())

    # Try to use the default input device
    try:
        audio = sd.rec(int(duration * sample_rate),
                       samplerate=sample_rate,
                       channels=1,
                       dtype="int16") # Using default device
    except sd.PortAudioError:
        print("❌ **Hata: Varsayılan ses cihazına erişilemiyor.**")
        print("Lütfen ses cihazı ayarlarınızı kontrol edin ve tekrar deneyin.")
        return None  # Kayıt başarısız olursa None döndür

    sd.wait()  # Kaydın bitmesini bekle
    print("✅ **Kayıt tamamlandı!**")

    # Kaydı bir .wav dosyasına kaydet
    wav.write("recorded_audio.wav", sample_rate, audio)

    return "recorded_audio.wav"

# Kaydı başlatmak için
audio_path = record_audio()

# Check if recording was successful
if audio_path:
    # Continue with transcription if recording was successful
    # ... (your transcription code) ...
    pass # Added a pass statement as a placeholder. Replace with your transcription code.
else:
    print("⚠️ **Ses kaydı başarısız oldu. Dönüştürme işlemi gerçekleştirilemiyor.**")

🔴 **Kayıt başladı... Konuşabilirsiniz!**

❌ **Hata: Varsayılan ses cihazına erişilemiyor.**
Lütfen ses cihazı ayarlarınızı kontrol edin ve tekrar deneyin.
⚠️ **Ses kaydı başarısız oldu. Dönüştürme işlemi gerçekleştirilemiyor.**


In [ ]:
import librosa

def transcribe_audio(audio_path):
    # Ses dosyasını yükleyelim
    audio, _ = librosa.load(audio_path, sr=16000)

    # Modelin beklediği formata çevirelim
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values

    # Modeli çalıştır ve tahmini al
    with torch.no_grad():
        logits = model(input_values).logits

    # Tahmin edilen en olası karakter dizisini al
    predicted_ids = torch.argmax(logits, dim=-1)

    # Modelin tahminini metne çevir
    transcription = processor.decode(predicted_ids[0])

    return transcription

# Kaydedilen sesi metne çevir
transcription_result = transcribe_audio(audio_path)

print("🔹 **Dönüştürülen Metin:**")
print(transcription_result)


TypeError: Invalid file: None

In [ ]:
from google.colab import widgets

button_ui = widgets.ToggleButtons(options=['Kayıtı Başlat', 'Kaydı Bitir'])

display(button_ui)


AttributeError: module 'google.colab.widgets' has no attribute 'ToggleButtons'

# Transformer ile IMDb Film Yorumları Üzerinde Duygu Analizi

In [ ]:
!pip install transformers torch datasets pandas matplotlib scikit-learn


In [ ]:
from datasets import load_dataset
import pandas as pd
from IPython.display import display

# IMDb veri setini yükleyelim
dataset = load_dataset("imdb")

# Veri setinin ilk 5 satırına bakalım
df_train = pd.DataFrame(dataset["train"]).sample(5)  # 5 rastgele örnek alalım

# 'ace_tools' yerine IPython.display kullanarak veri çerçevesini göster
display(df_train)
print("IMDb Veri Seti (Örnekler)") # Başlık ekleyebilirsiniz

,text,label
6868,"Dumb is as dumb does, in this thoroughly unint...",0
24016,I dug out from my garage some old musicals and...,1
9668,After watching this movie I was honestly disap...,0
13640,This movie was nominated for best picture but ...,1
14018,Just like Al Gore shook us up with his painful...,1


IMDb Veri Seti (Örnekler)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Modeli ve tokenizer'ı yükleyelim
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize edilmiş veriyi oluştur
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline

# Hazır duygu analizi modeli yükleyelim
sentiment_pipeline = pipeline("sentiment-analysis")

# Test yorumlarını deneyelim
test_sentences = [
    "This movie was absolutely amazing, I loved it!",
    "The plot was so boring and predictable.",
    "I had a great time watching this, highly recommended!",
    "Worst movie ever. Don't waste your time.",
]

# Modelin tahminlerini alalım
predictions = sentiment_pipeline(test_sentences)

import pandas as pd
# Mevcut olmayan 'ace_tools' yerine, IPython.display'den display fonksiyonunu kullanıyoruz
# ve özel bir başlık veriyoruz.
from IPython.display import display

df_predictions = pd.DataFrame({"Text": test_sentences, "Predicted Sentiment": [pred["label"] for pred in predictions]})
display(df_predictions)
print("Duygu Analizi Sonuçları (Hazır Model)") # Başlığı bu şekilde ekliyoruz

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


,Text,Predicted Sentiment
0,"This movie was absolutely amazing, I loved it!",POSITIVE
1,The plot was so boring and predictable.,NEGATIVE
2,"I had a great time watching this, highly recom...",POSITIVE
3,Worst movie ever. Don't waste your time.,NEGATIVE


Duygu Analizi Sonuçları (Hazır Model)


In [ ]:
import torch
import torch.nn as nn
import torchaudio

class AudioTransformer(nn.Module):
    def __init__(self, input_dim=16000, num_classes=10, num_heads=8, num_layers=6, d_model=256, dim_feedforward=512):
        super(AudioTransformer, self).__init__()

        self.embedding = nn.Linear(input_dim, d_model)  # 1D sinyali gömme katmanına dönüştür
        self.positional_encoding = nn.Parameter(torch.randn(1, input_dim, d_model))  # Pozisyon bilgisi ekle

        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=dim_feedforward),
            num_layers=num_layers
        )

        self.fc = nn.Linear(d_model, num_classes)  # Çıktıyı sınıflara dönüştür

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding  # Gömme işlemi ve pozisyon bilgisi
        x = self.transformer_encoder(x)  # Transformer Encoder'a giriş
        x = x.mean(dim=1)  # Ortalama alarak sınıflandırma için sabit boyutlu vektör elde et
        x = self.fc(x)  # Son çıktı
        return x

# Modeli oluştur
model = AudioTransformer()

# Örnek 1D ses sinyali (1 saniye, 16kHz)
sample_waveform = torch.randn(1, 16000)  # 1 saniyelik, 16kHz örnekleme hızında rastgele veri

# Modeli çalıştır
output = model(sample_waveform)
print(output.shape)  # (1, num_classes)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


torch.Size([1, 10])
